In [1]:
import tensorflow as tf
import sys
sys.path.append('../kitti_eval/flow_tool/')
sys.path.append('..')
import flowlib as fl

from nets.flow_net import Flow_net


from utils.loss_utils import SSIM, cal_grad2_error
from data_loader.data_loader import DataLoader
import matplotlib.pyplot as plt 
import os
import imageio
import time
from IPython.display import clear_output
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# tf.debugging.set_log_device_placement(True)

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [3]:
mode = "train_flow"
kitti_depth_folder = '../../../../datasets/kitti_3frames_256_832'

num_GPU = 2
EPOCH = 20
PER_BATCH = 4
BATCH_SIZE = PER_BATCH * num_GPU
IMG_HEIGHT = 256
IMG_WIDTH = 832
NUM_SCALE = 4
NUM_SOURCE = 2



In [4]:
checkpoint_attr = str(BATCH_SIZE) + '_' + str(IMG_HEIGHT) + '_' + str(IMG_WIDTH)
checkpoint_dir = os.path.join('../training_checkpoints', mode, checkpoint_attr)
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

# checkpoint = tf.train.Checkpoint(flowModel=model, flowOptimizer=optimizer)
# ckpt_manager = tf.train.CheckpointManager(checkpoint, directory=checkpoint_dir, checkpoint_name='ckpt', max_to_keep=10)

In [5]:
def decay(epoch):
    initial_learning_rate = 1e-4
    return initial_learning_rate * (0.5**(epoch//5))

In [6]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1, model.optimizer.lr.numpy()))

In [7]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='../logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [8]:
dataloader = DataLoader(mode=mode, 
                        dataset_dir=kitti_depth_folder, 
                        img_height=IMG_HEIGHT, 
                        img_width=IMG_WIDTH,
                        split='train')
dataset = dataloader.build_dataloader().shuffle(5000).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)



# strategy = tf.distribute.MirroredStrategy()
# dist_dataset = strategy.experimental_distribute_dataset(dataset)
    

In [9]:
# with strategy.scope():
model = Flow_net(input_shape=[IMG_HEIGHT, IMG_WIDTH, 9])
# tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [10]:
#model.add_loss(model.total_losses)

In [11]:
# with strategy.scope():
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=None)

In [12]:
print('# Fit model on training data')

model.fit(x=dataset, y=None, epochs=EPOCH, callbacks=callbacks)

# print('\nhistory dict:', history.history)

# Fit model on training data
Epoch 1/20
      1/Unknown - 299s 299s/step - loss: 3.6281WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (138.204067). Check your callbacks.
   5026/Unknown - 6961s 1s/step - loss: 4.2587
Learning rate for epoch 1 is 9.999999747378752e-05
   5026/Unknown - 6961s 1s/step - loss: 4.2587

InvalidArgumentError:  [_Derived_] 2 root error(s) found.
  (0) Invalid argument:  ConcatOp : Dimensions of inputs should match: shape[0] = [8,8,26,81] vs. shape[1] = [1,8,26,128]
	 [[{{node flow_net/get_flow/StatefulPartitionedCall/concat_2}}]]
	 [[flow_net/get_flow/StatefulPartitionedCall/Cast_11/_1766]]
  (1) Invalid argument:  ConcatOp : Dimensions of inputs should match: shape[0] = [8,8,26,81] vs. shape[1] = [1,8,26,128]
	 [[{{node flow_net/get_flow/StatefulPartitionedCall/concat_2}}]]
0 successful operations.
0 derived errors ignored.
	 [[cluster_0_1/partitioned_call]] [Op:__inference_distributed_function_107334]

Function call stack:
distributed_function -> distributed_function -> distributed_function -> distributed_function
